# Data Preprocessing Module MVP for Exploration

In this module I will be primarily focusing on the basics of preprocessing textual based data.
- Text Cleaning and Normalization
- Tokenization
- Deduplication
- Segmentation

In [1]:
# Importing libraries
import numpy as np
import pandas as pd
import os
import random
from datasets import load_dataset
from tqdm.notebook import tqdm

# Used for normalization and text cleaning
import re
import unicodedata

# For tokenizing
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize
from transformers import AutoTokenizer

/Users/rahul/miniconda3/envs/test_env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt_tab to /Users/rahul/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


### Text Cleaning and normalization
This step insures consistency in text input, reducing noise that can adversely affectlater preprocessing or model traning.

using the opensource [wikitext-103-raw-v1 dataset from huggingface](https://huggingface.co/datasets/iohadrubin/wikitext-103-raw-v1)

In [2]:
# Load the dataset
dataset = load_dataset("iohadrubin/wikitext-103-raw-v1", split="train")
df_wiki = pd.DataFrame(dataset)

Because the size of the preprocessed dataset was too large, I am capping the dataset size to 25 GB for initial analysis

In [3]:
# Calculate the byte size of each text entry
df_wiki['text_size'] = df_wiki['text'].apply(lambda x: len(x.encode('utf-8')))

# Compute cumulative size (in bytes)
df_wiki['cum_size'] = df_wiki['text_size'].cumsum()

# Define maximum allowed size: 25GB in bytes
max_bytes = 25 * 1024 * 1024 * 1024

# Filter the DataFrame to keep rows until we reach the cap
df_wiki_capped = df_wiki[df_wiki['cum_size'] <= max_bytes].copy()
print(f"Original dataset rows: {len(df_wiki)}")
print(f"Rows retained (capped to ~25GB): {len(df_wiki_capped)}")

# Drop the temporary columns
df_wiki_capped.drop(columns=['text_size', 'cum_size'], inplace=True)

Original dataset rows: 29567
Rows retained (capped to ~25GB): 29567


Data Exploration

In [4]:
df_wiki_capped.head()

,text
0,= Valkyria Chronicles III =\nSenjō no Valkyria...
1,= Tower Building of the Little Rock Arsenal =\...
2,= Cicely Mary Barker =\nCicely Mary Barker (28...
3,= Gambia women's national football team =\nThe...
4,= Plain maskray =\nThe plain maskray or brown ...


In [5]:
df_wiki_capped.tail()

,text
29562,"= Si Una Vez =\n""Si Una Vez"" (English: If I On..."
29563,= Sicklefin lemon shark =\nThe sicklefin lemon...
29564,= Flammulated flycatcher =\nThe flammulated fl...
29565,"= Ontario Highway 89 =\nKing's Highway 89, com..."
29566,= Luke Smith (writer) =\nLuke Michael Smith is...


In [6]:
# Basic info
print("Dataset columns:", df_wiki_capped.columns)
print("Dataset shape:", df_wiki_capped.shape)

Dataset columns: Index(['text'], dtype='object')
Dataset shape: (29567, 1)


#### Data cleaning exploration

In [7]:
# text length for each entry
df_wiki_capped['text_length'] = df_wiki_capped['text'].apply(lambda x: len(x))
print("Text lenght stats:")
print(df_wiki_capped['text_length'].describe())

Text lenght stats:
count     29567.000000
mean      17537.078161
std       14555.364685
min          16.000000
25%        7750.000000
50%       12994.000000
75%       22721.500000
max      140098.000000
Name: text_length, dtype: float64


In [8]:
# Lines with problematic formatting
problematic = df_wiki_capped[df_wiki_capped['text'].str.contains(r'["]{1,}', na=False)]
print("Rows with potential quotation issues: ") # decided to do this because of an table formatting error while doing df_wiki.head() step in the IDE.
# The error was: “Unterminated quoted field at end of CSV line”
print(problematic.head(5))

Rows with potential quotation issues: 
                                                text  text_length
0  = Valkyria Chronicles III =\nSenjō no Valkyria...        20297
1  = Tower Building of the Little Rock Arsenal =\...        20770
2  = Cicely Mary Barker =\nCicely Mary Barker (28...        15371
4  = Plain maskray =\nThe plain maskray or brown ...         6695
5  = 2011 – 12 Columbus Blue Jackets season =\nTh...        17189


## Data Contamination Simulator

This will artificially feed the contaminated data into the wiki-dataset for a artificial contamination simulation

In [9]:
# Load irrelevant text dataset
gutenberg_ds = load_dataset("sedthh/gutenberg_english", split="train")
gutenberg_sentences = gutenberg_ds['TEXT']

In [10]:
# Text contamination functions
def swap_words(text):
    words = text.split()
    if len(words) < 2:
        return text
    i, j = random.sample(range(len(words)), 2)
    words[i], words[j] = words[j], words[i]
    return ' '.join(words)

def add_char_noise(text, noise_level=0.05):
    def insert(s): return s[:i] + random.choice('abcdefghijklmnopqrstuvwxyz') + s[i:]
    def delete(s): return s[:i] + s[i+1:] if len(s) > 1 else s
    def substitute(s): return s[:i] + random.choice('abcdefghijklmnopqrstuvwxyz') + s[i+1:]
    def swap(s): return s[:i] + s[i+1] + s[i] + s[i+2:] if i < len(s)-1 else s
    ops = [insert, delete, substitute, swap]
    words = text.split()
    for w in range(len(words)):
        if random.random() < noise_level and len(words[w]) > 0:
            i = random.randint(0, len(words[w])-1)
            words[w] = random.choice(ops)(words[w])
    return ' '.join(words)

def insert_irrelevant_text(text):
    irrelevant = random.choice(gutenberg_sentences)
    words = text.split()
    insert_pos = random.randint(0, len(words))
    return ' '.join(words[:insert_pos] + irrelevant.split() + words[insert_pos:])

def contaminate_text(text):
    text = swap_words(text)
    text = add_char_noise(text)
    text = insert_irrelevant_text(text)
    return text

In [11]:
# Apply contamination to 20% of rows
contam_indices = df_wiki_capped.sample(frac=0.12, random_state=42).index
df_wiki_capped.loc[contam_indices, 'text'] = df_wiki_capped.loc[contam_indices, 'text'].apply(contaminate_text)

print(f"Contaminated {len(contam_indices)} rows using swap, noise, and Gutenberg insert.")

Contaminated 3548 rows using swap, noise, and Gutenberg insert.


In [12]:
# text length for each entry
df_wiki_capped['text_length'] = df_wiki_capped['text'].apply(lambda x: len(x))
print("Text lenght stats:")
print(df_wiki_capped['text_length'].describe())

Text lenght stats:
count    2.956700e+04
mean     5.877762e+04
std      1.660840e+05
min      1.600000e+01
25%      8.355500e+03
50%      1.492900e+04
75%      3.039400e+04
max      4.892107e+06
Name: text_length, dtype: float64


In [13]:
# Lines with problematic formatting
problematic = df_wiki_capped[df_wiki_capped['text'].str.contains(r'["]{1,}', na=False)]
print("Rows with potential quotation issues: ") # decided to do this because of an table formatting error while doing df_wiki.head() step in the IDE.
# The error was: “Unterminated quoted field at end of CSV line”
print(problematic.head(5))

Rows with potential quotation issues: 
                                                text  text_length
0  = Valkyria Chronicles III =\nSenjō no Valkyria...        20297
1  = Tower Building of the Little Rock Arsenal =\...        20770
2  = Cicely Mary Barker =\nCicely Mary Barker (28...        15371
4  = Plain maskray =\nThe plain maskray or brown ...         6695
5  = 2011 – 12 Columbus Blue Jackets season =\nTh...        17189


### Text Cleaning and Normalization
cleaning function to:
- Remove HTML tags
- Normalize Unicode to standardize characters.
- Convert text to lowercase and remove accent marks
- Remove non-UTF characters and extra whitespaces.

This should make the text more uniform for intial tokenization.

In [14]:
def normalize_text(text):
    # Remove HTML tags
    text = re.sub(r'<.*?>', ' ', text)
    # Normalize Unicode (NFC)
    text = unicodedata.normalize('NFC', text)
    # Lowercase conversion and accent stripping
    text = text.lower()
    text = ''.join(c for c in text if not unicodedata.combining(c))
    # Remove non-UTF characters and extra whitespace
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [15]:
# Applying normalizations
df_wiki_capped['cleaned_text'] = df_wiki_capped['text'].apply(normalize_text)
print("cleaned sample:")
print(df_wiki_capped[['text', 'cleaned_text']].head(5))

cleaned sample:
                                                text  \
0  = Valkyria Chronicles III =\nSenjō no Valkyria...   
1  = Tower Building of the Little Rock Arsenal =\...   
2  = Cicely Mary Barker =\nCicely Mary Barker (28...   
3  = Gambia women's national football team =\nThe...   
4  = Plain maskray =\nThe plain maskray or brown ...   

                                        cleaned_text  
0  = valkyria chronicles iii = senj no valkyria 3...  
1  = tower building of the little rock arsenal = ...  
2  = cicely mary barker = cicely mary barker (28 ...  
3  = gambia women's national football team = the ...  
4  = plain maskray = the plain maskray or brown s...  


### Tokenization
Tokenizing the cleaned data using Hugging Face fast tokenizer for essential downstream processing

In [16]:
# Tokenizer initialization
# tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', use_fast=True)
tokenizer = AutoTokenizer.from_pretrained('distilgpt2', use_fast=True)


def tokenize_text(text):
    tokens = tokenizer.tokenize(text, truncation=True, max_length=1024) # changes made because of warning
    return tokens

df_wiki_capped['tokens'] = df_wiki_capped['cleaned_text'].apply(tokenize_text)
print("tokenized sample:")
print(df_wiki_capped[['cleaned_text', 'tokens']].head(2))

tokenized sample:
                                        cleaned_text  \
0  = valkyria chronicles iii = senj no valkyria 3...   
1  = tower building of the little rock arsenal = ...   

                                              tokens  
0  [=, Ġv, alky, ria, Ġchron, icles, Ġiii, Ġ=, Ġs...  
1  [=, Ġtower, Ġbuilding, Ġof, Ġthe, Ġlittle, Ġro...  


### Deduplication
Deduplication is the process of removing duplicates or near-duplicates to avoid redundancy in the dataset. For now I am choosing to remove the exact-matches.

In [17]:
def duplicate_texts(df, text_column = 'cleaned_text'):
    df = df.drop_duplicates(subset=[text_column])
    return df

df_wiki_capped_unique = duplicate_texts(df_wiki_capped)
print("Number of rows after deduplication:", len(df_wiki_capped_unique))

Number of rows after deduplication: 29224


### Data Segmentation
segmenting the cleaned text into sentences for finer analysis using NLTK's sentence tokenizer.

In [18]:
def segment_text(text, mode='sentence', fixed_token_length=100):
    if mode == 'sentence':
        segments = sent_tokenize(text)
    elif mode == 'fixed':
        tokens = tokenize_text(text)
        segments = [' '.join(tokens[i:i+fixed_token_length]) for i in range(0, len(tokens), fixed_token_length)]
    else:
        segments = [text]
    return segments

In [19]:
def segment_dataframe(df, text_column='cleaned_text', mode='sentence'):
    df['segments'] = df[text_column].apply(lambda x: segment_text(x, mode=mode))
    df_segmented = df.explode('segments')
    return df_segmented

In [20]:
df_segmented = segment_dataframe(df_wiki_capped_unique, text_column='cleaned_text', mode='sentence')
print("Segmented data sample:")
print(df_segmented[['cleaned_text', 'segments']].head(5))

/var/folders/3j/w0qrc74d6sj7fjdwc593xcqc0000gn/T/ipykernel_45186/2733018757.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['segments'] = df[text_column].apply(lambda x: segment_text(x, mode=mode))


Segmented data sample:
                                        cleaned_text  \
0  = valkyria chronicles iii = senj no valkyria 3...   
0  = valkyria chronicles iii = senj no valkyria 3...   
0  = valkyria chronicles iii = senj no valkyria 3...   
0  = valkyria chronicles iii = senj no valkyria 3...   
0  = valkyria chronicles iii = senj no valkyria 3...   

                                            segments  
0  = valkyria chronicles iii = senj no valkyria 3...  
0  valkyria of the battlefield 3), commonly refer...  
0  released in january 2011 in japan, it is the t...  
0  employing the same fusion of tactical and real...  
0  the game began development in 2010, carrying o...  


In [21]:
len(df_segmented)

14938261

In [22]:
df_segmented.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14938261 entries, 0 to 29566
Data columns (total 5 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   text          object
 1   text_length   int64 
 2   cleaned_text  object
 3   tokens        object
 4   segments      object
dtypes: int64(1), object(4)
memory usage: 683.8+ MB


In [23]:
# Dropping the original text column as we don't need that for further modules
df_segmented.drop(columns=['text', 'cleaned_text'], inplace=True)

In [24]:
df_segmented.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14938261 entries, 0 to 29566
Data columns (total 3 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   text_length  int64 
 1   tokens       object
 2   segments     object
dtypes: int64(1), object(2)
memory usage: 455.9+ MB


In [25]:
# Further capping the data
# Calculate total number of segmented rows
total_rows = len(df_segmented)
one_tenth_rows = int(total_rows / 10)
print(f"Total segmented rows: {total_rows}")
print(f"Keeping only a quater: {one_tenth_rows} rows")

Total segmented rows: 14938261
Keeping only a quater: 1493826 rows


In [26]:
# Select the first one third of the rows
df_segmented_subset = df_segmented.iloc[:one_tenth_rows].copy()

In [27]:
df_segmented_subset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1493826 entries, 0 to 3047
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   text_length  1493826 non-null  int64 
 1   tokens       1493826 non-null  object
 2   segments     1493826 non-null  object
dtypes: int64(1), object(2)
memory usage: 45.6+ MB


### Saving the preprocessed data

In [28]:
output_dir = "../data"
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, "preprocessed_wikitext103_subset.csv")

# Save the segmented DataFrame to the specified CSV file
df_segmented_subset.to_csv(output_file, index=False)
print(f"Subset of preprocessed data saved to {output_file}")

Subset of preprocessed data saved to ../data/preprocessed_wikitext103_subset.csv


In [30]:
# testing by loading the data

# Define the file path in the "data" directory
data_file = os.path.join("../data", "preprocessed_wikitext103_subset.csv")

# loading data in chunks as the data size is massive
with open(data_file, 'r', encoding='utf-8', errors='ignore') as f:
    total_lines = sum(1 for line in f) - 1

chunk_size = 10000
total_chunks = total_lines // chunk_size

# Read CSV in chunks and display progress
chunks = []
for chunk in tqdm(pd.read_csv(data_file, on_bad_lines='skip', engine='python', chunksize=chunk_size),
                  total=total_chunks,
                  desc="Loading CSV"):
    chunks.append(chunk)

# Combine all chunks into one DataFrame
df = pd.concat(chunks, ignore_index=True)

# Display a sample of the loaded data and its dimensions
print("Loaded data sample:")
print(df.head())
print("\nShape of loaded data:", df.shape)

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html